<a target="_blank" href="https://colab.research.google.com/github/jackma-00/house-price-prediction/blob/main/experiments/1_house_price_feature_backfill.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Feature Backfill

Install required packages

In [ ]:
!pip install hopsworks[python]

## Imports

In [ ]:
import pandas as pd
import hopsworks
import os

## Connect to Hopsworks

In [ ]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = ""
proj = hopsworks.login()


Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1158295


## Unzip and read CSV into DataFrame

In [ ]:
# Connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# unzip properties.zip
!unzip /content/drive/MyDrive/properties.zip -d /content/

Archive:  /content/drive/MyDrive/properties.zip
  inflating: /content/properties.csv  


In [ ]:
# unzip omi.zip
!unzip /content/drive/MyDrive/omi.zip -d /content/

Archive:  /content/drive/MyDrive/omi.zip
  inflating: /content/omi.csv        


In [ ]:
properties_csv = 'properties.csv'
omi_csv = 'omi.csv'

properties_df = pd.read_csv(properties_csv)
omi_df = pd.read_csv(omi_csv)

In [ ]:
properties_df

,id,typologyId,categoryCode,codcom,agencyId,latitude,longitude,id_zona_omi,city,zona,...,GA4Heating,hasElevator,isNew,isLuxury,description,photos,scraping_date,createdAt,updatedAt,categoryName
0,109230993,14,1,1272.0,NaN,45.082779,7.644110,L219-D8,Torino,"Pozzo Strada, Parella",...,Centralizzato,NaN,True,False,Bilocale pronto da abitare situato a 5 minuti ...,"[{'id': 1471294881, 'caption': '', 'urls': {'s...",2024-03-18,1.707222e+09,1.709585e+09,NaN
1,108895067,14,1,1272.0,178139.0,45.083300,7.645500,L219-D8,Torino,"Pozzo Strada, Parella",...,Centralizzato,NaN,False,False,ZONA CAMPIDOGLIO - VIA MICHELE LESSONA - In pi...,"[{'id': 1465033121, 'caption': 'facciata', 'ur...",2024-03-18,1.706099e+09,1.709984e+09,NaN
2,109784457,14,1,1272.0,NaN,45.058979,7.683780,L219-B8,Torino,San Salvario,...,Centralizzato,NaN,True,False,"Piazza madama, a 5 minuti dal Parco del Valent...","[{'id': 1481631325, 'caption': '', 'urls': {'s...",2024-03-18,1.709151e+09,1.710337e+09,NaN
3,108547953,14,1,1272.0,392458.0,45.057200,7.678300,L219-B8,Torino,San Salvario,...,Autonomo,NaN,False,False,MONOLOCALE IN AFFITTO ARREDATO\nRISCALDAMENTO ...,"[{'id': 1458345959, 'caption': '', 'urls': {'s...",2024-03-18,1.704902e+09,1.710408e+09,NaN
4,109914147,14,1,23059.0,229787.0,45.449700,10.668200,G489-B2,Peschiera del Garda,Centro,...,Autonomo,NaN,True,False,Per informazioni ELENA 393/8205511\nIn zona Sa...,"[{'id': 1484040585, 'caption': '', 'urls': {'s...",2024-03-25,1.709641e+09,1.709642e+09,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211904,110000113,17,1,1272.0,74278.0,45.090100,7.698500,L219-C10,Torino,"Aurora, Barriera di Milano, Rebaudengo",...,Autonomo,NaN,True,False,In via Saverio Mercadante affittiamo partico...,"[{'id': 1485646521, 'caption': 'cortile', 'url...",2024-03-18,1.709894e+09,1.709895e+09,NaN
1211905,109942307,14,1,1272.0,NaN,45.092331,7.699990,L219-D12,Torino,"Aurora, Barriera di Milano, Rebaudengo",...,Centralizzato,NaN,True,False,"Alloggio in buone condizioni, abitabile, espos...","[{'id': 1484556023, 'caption': '', 'urls': {'s...",2024-03-18,1.709718e+09,1.710158e+09,NaN
1211906,108302933,14,1,1272.0,NaN,45.091194,7.696717,L219-C10,Torino,"Aurora, Barriera di Milano, Rebaudengo",...,Autonomo,NaN,False,False,"Privato affitta trilocale di 65 mq , zona Barr...","[{'id': 1452992687, 'caption': '', 'urls': {'s...",2024-03-18,1.703504e+09,1.704724e+09,NaN
1211907,110019089,14,1,48017.0,247117.0,43.766900,11.251600,D612-B5,Firenze,Oltrarno,...,Autonomo,NaN,True,False,This wonderful 1BR apartment is in a fantastic...,"[{'id': 1486030391, 'caption': 'Interno 1', 'u...",2024-03-15,1.709841e+09,1.710492e+09,NaN


In [ ]:
omi_df

,id_zona_omi,tipologia,condizione,compr_min,compr_max,comune_amm,zona_omi_type,nome_zona,provincia,area_territoriale,regione,geometry
0,A048-B1,Abitazioni civili,normale,812.0,1165.0,A048,B,ACQUAVIVA DELLE FONTI - Zona OMI B1,BA,SUD,PUGLIA,"POLYGON ((16.845568 40.897414, 16.844439 40.89..."
1,A048-B1,Abitazioni civili,ottimo,1062.0,1454.0,A048,B,ACQUAVIVA DELLE FONTI - Zona OMI B1,BA,SUD,PUGLIA,"POLYGON ((16.845568 40.897414, 16.844439 40.89..."
2,A048-B1,Abitazioni civili,scadente,539.0,787.0,A048,B,ACQUAVIVA DELLE FONTI - Zona OMI B1,BA,SUD,PUGLIA,"POLYGON ((16.845568 40.897414, 16.844439 40.89..."
3,A048-B1,Abitazioni di tipo economico,normale,650.0,930.0,A048,B,ACQUAVIVA DELLE FONTI - Zona OMI B1,BA,SUD,PUGLIA,"POLYGON ((16.845568 40.897414, 16.844439 40.89..."
4,A048-B1,Abitazioni di tipo economico,ottimo,892.0,1192.0,A048,B,ACQUAVIVA DELLE FONTI - Zona OMI B1,BA,SUD,PUGLIA,"POLYGON ((16.845568 40.897414, 16.844439 40.89..."
...,...,...,...,...,...,...,...,...,...,...,...,...
245119,M422-E1,Abitazioni di tipo economico,ottimo,787.0,918.0,M422,E,PIEVE DEL GRAPPA - Zona OMI E1,TV,NORD-EST,VENETO,"POLYGON ((11.841451 45.818366, 11.841574 45.81..."
245120,M422-E1,Abitazioni di tipo economico,scadente,328.0,412.0,M422,E,PIEVE DEL GRAPPA - Zona OMI E1,TV,NORD-EST,VENETO,"POLYGON ((11.841451 45.818366, 11.841574 45.81..."
245121,M422-E1,Ville e Villini,normale,680.0,832.0,M422,E,PIEVE DEL GRAPPA - Zona OMI E1,TV,NORD-EST,VENETO,"POLYGON ((11.841451 45.818366, 11.841574 45.81..."
245122,M422-E1,Ville e Villini,ottimo,920.0,1050.0,M422,E,PIEVE DEL GRAPPA - Zona OMI E1,TV,NORD-EST,VENETO,"POLYGON ((11.841451 45.818366, 11.841574 45.81..."


## Data Cleaning

In [ ]:
properties_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1211909 entries, 0 to 1211908
Data columns (total 44 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   id                   1211909 non-null  int64  
 1   typologyId           1211909 non-null  int64  
 2   categoryCode         1211909 non-null  int64  
 3   codcom               1209642 non-null  float64
 4   agencyId             1110765 non-null  float64
 5   latitude             1211909 non-null  float64
 6   longitude            1211909 non-null  float64
 7   id_zona_omi          1206438 non-null  object 
 8   city                 1211710 non-null  object 
 9   zona                 915626 non-null   object 
 10  quartiere            438259 non-null   object 
 11  name                 1211909 non-null  object 
 12  contract             1211909 non-null  object 
 13  surface              1211909 non-null  int64  
 14  condition            1120461 non-null  object 
 15

In [ ]:
omi_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245124 entries, 0 to 245123
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   id_zona_omi        245124 non-null  object 
 1   tipologia          245124 non-null  object 
 2   condizione         245124 non-null  object 
 3   compr_min          245124 non-null  float64
 4   compr_max          245124 non-null  float64
 5   comune_amm         245124 non-null  object 
 6   zona_omi_type      245124 non-null  object 
 7   nome_zona          245124 non-null  object 
 8   provincia          240570 non-null  object 
 9   area_territoriale  245124 non-null  object 
 10  regione            245124 non-null  object 
 11  geometry           245124 non-null  object 
dtypes: float64(2), object(10)
memory usage: 22.4+ MB


In [ ]:
properties_df.dropna(inplace=True)
omi_df.dropna(inplace=True)

## Create the Feature Groups and insert the DataFrames in them

In [ ]:
fs = proj.get_feature_store()

In [ ]:
# Get or create the 'properties' feature group
properties_fg = fs.get_or_create_feature_group(
    name='properties',
    description='Properties characteristics and respective prices',
    version=2,
    primary_key=['id'],
    online_enabled=True,
)

In [ ]:
# Insert data into feature group
properties_fg.insert(properties_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1158295/fs/1148998/fg/1391994


Uploading Dataframe: 100.00% |██████████| Rows 11232/11232 | Elapsed Time: 00:04 | Remaining Time: 00:00


Launching job: properties_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1158295/jobs/named/properties_2_offline_fg_materialization/executions


(Job('properties_2_offline_fg_materialization', 'SPARK'), None)

In [ ]:
omi_fg = fs.get_or_create_feature_group(
    name='omi',
    description='OMI information',
    version=1,
    primary_key=['id_zona_omi'],
    online_enabled=True,
)

In [ ]:
# Insert data into feature group
omi_fg.insert(omi_df)

Uploading Dataframe: 100.00% |██████████| Rows 240570/240570 | Elapsed Time: 00:54 | Remaining Time: 00:00


Launching job: omi_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1158295/jobs/named/omi_1_offline_fg_materialization/executions


(Job('omi_1_offline_fg_materialization', 'SPARK'), None)